<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/(%EB%8B%A4%EC%9D%B8%EC%9A%A9)%20F_%EC%A6%9D%EB%B6%84%20%EB%8B%A4%EC%8B%9C%ED%95%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		   df_A_inc.csv   df_O_inc.csv
 A_inc_model.pt		   df_B_inc.csv   df_Q_inc.csv
 B_inc_model.pt		   df_D_inc.csv   df_R_inc.csv
 classes.txt		   df_E_inc.csv   E_inc_model.pt
'Colab Notebooks'	   df_J_inc.csv   labels.zip
'답안 작성용 파일.csv'	   df_K_inc.csv  'My Drive'
 dabunru_inc_model_22.pt   df_M_inc.csv   spell_check
 dabunru_inc_model.pt	   df_N_inc.csv   val.txt


In [10]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/spell_check/F_spell_check.csv')

## 데이터 전처리

In [11]:
df=df.reset_index(drop=True)

In [12]:
df['digit_3'].value_counts()

424    11152
422     5744
421     4724
423     4360
411     3582
412     3363
426     1566
425      559
Name: digit_3, dtype: int64

In [13]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [14]:
temp_result=[]
n=0
for sentence in df1['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=5)
  id_list=['p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n])]
  digit_1_list=[df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n]]
  digit_2_list=[df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n]]
  digit_3_list=[df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df1_to_insert = pd.DataFrame(data_to_insert)
  df1= df1.append(df1_to_insert, ignore_index=True)
  n=n+1

기존 데이터 합치기

In [15]:
df=pd.concat([df, df1])

In [16]:
df1

,AI_id,digit_1,digit_2,digit_3,text
0,id_0001765,F,42,425,"사업장에서 고객의 의뢰를 받아 전문건설하도급 (시설물 유지관리 공사, 상하수도)"
1,id_0001962,F,42,425,고객의 사업장에서 고객 요청에 따라 시설물 유지관리 공사
2,id_0002818,F,42,425,건설현장에서 고객의 의뢰를 받아 토목시설 건설
3,id_0003478,F,42,425,현장에서 고객의 요청으로 시설물 유지공사
4,id_0003751,F,42,425,사업체에서 시설을 갖추고 난방보일러 유지 보수
...,...,...,...,...,...
38411,pid_0999993,F,41,412,고객의 조경건설공사 고객 요구의 따라 사업장에서
38412,pid_0999993,F,41,412,고객의 사업장에서 고객 요구의 따라 조경건설공사
38413,pid_0999993,F,41,412,고객의 사업장에서 고객 요구의 따라 조경건설공사
38414,pid_0999993,F,41,412,고객의 사업장에서 고객 요구의 따라 조경건설공사


In [17]:
df['digit_3'].value_counts()

412    26904
426    12528
424    11152
422     5744
421     4724
425     4472
423     4360
411     3582
Name: digit_3, dtype: int64

In [18]:
df.loc[(df['digit_3'] == 411), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 412), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 421), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 422), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 423), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 424), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 425), 'digit_3'] = 6  
df.loc[(df['digit_3'] == 426), 'digit_3'] = 7  

In [19]:
df.to_csv('df_F_inc.csv', index=False, encoding='utf-8-sig')